In [12]:
import fastf1
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

In [ ]:
fastf1.Cache.enable_cache('fastf1_cache')

In [ ]:
qualifying = fastf1.get_session(2024, 'Monaco', 'Q')
qualifying.load()
qualifying=qualifying.results
res=pd.DataFrame()
res['driver_name']=qualifying['DriverId']
predict_data = pd.DataFrame({'Position_Q': range(1, len(res) + 1)})
predict_data=model.predict(predict_data)
res['prediction']=predict_data
res.sort_values(by="prediction")

In [27]:
class Jesus_pls_work:
    def __init__(self):
        self.finaldf = pd.DataFrame()
    
    def datagather(self,year,loc):
        quali=fastf1.get_session(year,loc,'Q')
        race=fastf1.get_session(year,loc,'R')
        
        quali.load(weather=True)
        race.load(weather=True)
        
        weather=quali.weather_data
        qavgtemp=weather['AirTemp'].mean()
        qrainfall=weather['Rainfall'].sum()

        # laps = quali.laps
        # qyellow_flags = laps['YellowFlag'].sum()
        # qred_flags = laps['RedFlag'].sum()
        
        weather=race.weather_data
        ravgtemp=weather['AirTemp'].mean()
        rrainfall=weather['Rainfall'].sum()
        
        # laps = race.laps
        # ryellow_flags = laps['YellowFlag'].sum()
        # rred_flags = laps['RedFlag'].sum()

        qresults = quali.results[['DriverNumber', 'Position']]
        rresults = race.results[['DriverNumber', 'Position']]

        qualidata= pd.DataFrame({
            'DriverNumber': qresults['DriverNumber'],
            'AvgTemperature': qavgtemp,
            'TotalRainfall': qrainfall,
            # 'YellowFlags': qyellow_flags,
            # 'RedFlags': qred_flags,
            'Position': qresults['Position']  
        })
        
        racedata= pd.DataFrame({
            'DriverNumber': rresults['DriverNumber'],
            'AvgTemperature': ravgtemp,
            'TotalRainfall': rrainfall,
            # 'YellowFlags': ryellow_flags,
            # 'RedFlags': rred_flags,
            'Position': rresults['Position']  
        })

        self.finaldf = qualidata.merge(racedata, on='DriverNumber', suffixes=('_Q', '_R'))

    def dataprocessing(self):
        df=self.finaldf
        self.finaldf['DriverNumber'] = pd.to_numeric(self.finaldf['DriverNumber'])
        print(df.head())
        y=df['Position_R']
        x=df.drop('Position_R',axis=1)
        
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=420)
        
        model = xgb.XGBRegressor(objective='reg:squarederror', learning_rate=0.1, max_depth=5, n_estimators=100)
        model.fit(X_train, y_train)

        preds = model.predict(X_test)
        print(f"Mean Absolute Error: {mean_absolute_error(y_test, preds)}")    

In [28]:
obt=Jesus_pls_work()
obt.datagather(year=2023,loc='Monaco')
obt.dataprocessing()

logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	Using cached data for season_schedule
core           INFO 	Loading data for Monaco Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '55', '4', '10', '44', '18', '31', '27', '14', '81', '11', '63', '24', '21', '22', '77', '20', '

   DriverNumber  AvgTemperature_Q  TotalRainfall_Q  Position_Q  \
0             1         25.586047                0         1.0   
1            55         25.586047                0         2.0   
2             4         25.586047                0         3.0   
3            10         25.586047                0         4.0   
4            44         25.586047                0         5.0   

   AvgTemperature_R  TotalRainfall_R  Position_R  
0         25.065909               26         1.0  
1         25.065909               26         8.0  
2         25.065909               26         9.0  
3         25.065909               26         7.0  
4         25.065909               26         4.0  
Mean Absolute Error: 3.7142768502235413
